<a href="https://colab.research.google.com/github/malihazahid/agentic-AI/blob/main/Project_2_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00


In [51]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('pinecone')

pc = Pinecone(api_key=pinecone_api_key)

In [52]:
index_name = "online-rag-project"

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'f8accd5097502f68b15e02a0d282be91', 'Date': 'Mon, 06 Jan 2025 16:52:30 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Correct environment variable name
os.environ['GOOGLE_API_KEY'] = userdata.get('secretName')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("We are builing a RAG project")

In [ ]:
vector[:5]

[0.02158137783408165,
 -0.029227618128061295,
 -0.06158623844385147,
 -0.01337017584592104,
 -0.03250042721629143]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['fa805153-d046-4e40-bc7b-1ab910821b2c',
 '51b28738-9d0a-4f3e-b93c-ee4017c3f5c4',
 '8d164974-dbfc-4edc-80bd-7a911ed1be01',
 '71870f24-a2f3-4d6e-ae97-3bf96322d3d7',
 '5186735b-a8a6-49c6-b3d3-5183e9cf25ac',
 'b11d0df2-6aed-4208-bc94-03a252a45805',
 '561a8928-aa59-466f-884f-74a64c9a3c9a',
 '3b71c53b-8098-49fb-9dea-7e0fbf9ee71d',
 'ce7e8df3-5096-4738-bca1-59c557df67f0',
 'a4233522-be41-4233-886a-cdd0821c1b49']

In [ ]:
# data retrieval
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [40]:
! pip install langchain langchain_google_genai

In [41]:
import langchain_google_genai as genai

In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [43]:
from google.colab import userdata
api_key = userdata.get('secretName')

In [64]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Change model_name to model
    api_key=api_key,
    temperature=0.3,
)

In [66]:
# RAG Chain
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=vector_store.as_retriever())

def answer_to_user(query: str) -> str:
  """Answers a user query using RAG."""
  result = qa.run(query)
  return result
# Example usage:
queries = [
    "What is the weather like tomorrow?",
    "What is LangChain used for?",
    "Tell me about the bank robbery.",
    "What do people tweet about?",
    "What is LangGraph?",
    "Will it be hot tomorrow?",
    "What is the stock market doing?"
]

for query in queries:
  answer = answer_to_user(query)
  print(f"Question: {query}")
  print(f"Answer: {answer}\n")


Question: What is the weather like tomorrow?
Answer: The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.


Question: What is LangChain used for?
Answer: Based on the provided text, LangChain is used for building exciting new projects.  More information is needed to give a more complete answer.


Question: Tell me about the bank robbery.
Answer: Robbers broke into the city bank and stole $1 million in cash.


Question: What do people tweet about?
Answer: Based on the provided text, I don't know what people tweet about.  The text only expresses a repeated feeling of impending deletion.


Question: What is LangGraph?
Answer: Based on the provided text, LangGraph is a framework for building stateful, agentic applications.


Question: Will it be hot tomorrow?
Answer: No, the high will be 62 degrees.  The forecast is cloudy and overcast.


Question: What is the stock market doing?
Answer: The stock market is down 500 points today due to fears of a recession.
